## Reproducing result figures and evaluation metrics

This notebook can be used to produce the different evaluation metrics and associated figures: 

* Evolution of kinetic energy over time
* Energy spectrum as a function of the azimuthal wavenumber
* Radial profiles of azimuthal velocity and potential vorticity
* Zonal and residual energy spectrum based on the Bessel-Fourier transform
* Hovmöller maps of the velocity components

The notebook can also produce the statistical evaluation based on integrated metrics:

* $Re$: Reynolds number
* $E_{Z} / E_{T}$: Zonal energy ratio
* $\mathcal{Z}$: Enstrophy
* $\ell_{\nu}$: Dissipation lengthscale
* $\mathcal{P}_\mathcal{F}$: Injected power
* $\alpha_\nu^{\Upsilon}$: Ratio of dissipative processes

In [ ]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import tqdm
import h5py
import matplotlib.pyplot as plt
import matplotlib.lines as plt_lines

plt.rcParams.update({
  'mathtext.fontset': 'cm'
})

import numpy as np
import scipy
import jax
import jax.numpy as jnp
import jax.random as jnr

jax.config.update(
  'jax_enable_x64', True
)

import models.imex_solver as imex
from models.qg_annulus import (
    QgAnnulus, 
    dynamical_solver,
    cartesian_forcing,
    average,
    reynolds,
    azimuthal_spectrum,
    hankel_spectrum
)
from utils import (
    from_m,
    quad_r,
    coef_r,
    hankel_kernels
)

In [ ]:
save_path = '' # your save path (as used in eval.py)

### Alternative models - kinetic energy evolutions and azimuthal spectra $E_{T}(m)$:

In [ ]:
# Loading configuration
cfg_name = 'iii'
cfg_data = os.path.join(save_path, cfg_name)
cfg_path = os.path.join('../data', cfg_name)
eq, *_ = QgAnnulus.load(os.path.join(cfg_path, 'snapshot.h5'))

# Dataset (and associated evaluation) name
data_name = 'continuous-turnover'
with h5py.File(os.path.join(cfg_path, data_name + '_dataset.h5'), 'r') as f:
    coarse_factor = f.attrs['coarse_factor']

eq_coarse = QgAnnulus(
    E=eq.E,
    cte_beta=eq.cte_beta,
    radius_ratio=eq.s_i / eq.s_o,
    n_m=int((eq.n_m - 1) / coarse_factor) + 1,
    n_s=int((eq.n_s - 1) / coarse_factor) + 1
)

def alt_models_stats(
    name: str,
    file_path: str,
    eq: QgAnnulus
):
    comp_path = file_path + '.alt_models_stats.npz'
    if os.path.isfile(comp_path):
        comp_data = np.load(comp_path)
        return (
            comp_data['time'],
            comp_data['total_ke_t'],
            comp_data['zonal_ke_t'],
            comp_data['m_spectra']
        )
    else:
        total_ke_t = []
        zonal_ke_t = []
        m_spectra = []
        
        with h5py.File(file_path, 'r') as f:
            time = np.array(f['time'])
            samples = len(time)
            samples_digits = len(str(samples))
            print('Computing kinetic energy evolution and azimuthal spectrum for {} ({} samples)'.format(name, str(samples)))
            pbar = tqdm.tqdm(range(samples), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
            for i in pbar:
                i_str = str(i).zfill(samples_digits)
                us_m = np.array(f['us_m_' + i_str])
                up_m = np.array(f['up_m_' + i_str])
    
                total_ke_t.append(
                    0.5 * (average(eq, us_m) + average(eq, up_m))
                )
                zonal_ke_t.append(
                    0.5 * average(eq, up_m[[0]])
                )
                m_spectra.append(
                    (azimuthal_spectrum(eq, us_m) + azimuthal_spectrum(eq, up_m)) / eq.surf
                )
        np.savez(
            comp_path, 
            time=time, 
            total_ke_t=total_ke_t, 
            zonal_ke_t=zonal_ke_t, 
            m_spectra=m_spectra
        )
        return (
            time,
            np.array(total_ke_t),
            np.array(zonal_ke_t),
            np.array(m_spectra)
        )

# Reference
path_dns = os.path.join(cfg_data, data_name + '_eval_dns.h5')
time_dns, total_ke_t_dns, zonal_ke_t_dns, m_spectra_dns = alt_models_stats(
    name='DNS', 
    file_path=path_dns, 
    eq=eq
)

# Models
path_learn = os.path.join(cfg_data, data_name + '_eval_learn.h5')
time_learn, total_ke_t_learn, zonal_ke_t_learn, m_spectra_learn = alt_models_stats(
    name='`Learned` model', 
    file_path=path_learn, 
    eq=eq_coarse
)

path_hdiff = os.path.join(cfg_data, data_name + '_eval_hdiff.h5')
time_hdiff, total_ke_t_hdiff, zonal_ke_t_hdiff, m_spectra_hdiff = alt_models_stats(
    name='`Hyperdiffusivity` model', 
    file_path=path_hdiff, 
    eq=eq_coarse
)

path_0 = os.path.join(cfg_data, data_name + '_eval_0.h5')
time_0, total_ke_t_0, zonal_ke_t_0, m_spectra_0 = alt_models_stats(
    name='`Under-resolved` model',
    file_path=path_0,     
    eq=eq_coarse
)

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(7.5, 4.0), dpi=120)

axs[0].plot(time_dns,   total_ke_t_dns,   label=r'$\text{DNS}$', color='k')
axs[0].plot(time_learn, total_ke_t_learn, label=r'$\tau \equiv \mathcal{M}$', color='tab:blue')
axs[0].plot(time_hdiff, total_ke_t_hdiff, label=r'$\tau \equiv d(m)$', color='tab:orange')
axs[0].plot(time_0,     total_ke_t_0,     label=r'$\tau \equiv 0$', color='tab:green')

axs[0].plot(time_dns,   zonal_ke_t_dns,   alpha=0.6, linestyle='--', color='k')
axs[0].plot(time_learn, zonal_ke_t_learn, alpha=0.6, linestyle='--', color='tab:blue')
axs[0].plot(time_hdiff, zonal_ke_t_hdiff, alpha=0.6, linestyle='--', color='tab:orange')
axs[0].plot(time_0,     zonal_ke_t_0,     alpha=0.6, linestyle='--', color='tab:green')

axs[0].axvspan(time_dns[0], time_0[0], color='darkgrey', alpha=0.3)
axs[0].ticklabel_format(useMathText=True)
axs[0].set_xlabel(r'$t$', fontsize=15)
axs[0].set_xticks([0.01, 0.015, 0.02, 0.025])
axs[0].set_ylabel(r'$\text{Kinetic \,\, energy}$', fontsize=15)
axs[0].set_xlim(right=time_dns[-1])
axs[0].set_ylim(bottom=0, top=4.5e7)
E_T = plt_lines.Line2D([], [], color='k', label=r'$E_T$')
E_Z = plt_lines.Line2D([], [], color='k', linestyle='--', alpha=0.2, label=r'$E_Z$')
axs[0].legend(handles=[E_T, E_Z], fontsize=13, frameon=False)
axs[0].tick_params(reset=True, axis='y', which='both', direction='in')

axs[1].loglog(np.arange(eq.n_m) + 1,        np.mean(m_spectra_dns,   axis=0), label=r'$\text{DNS}$', color='k')
axs[1].loglog(np.arange(eq_coarse.n_m) + 1, np.mean(m_spectra_learn, axis=0), label=r'$\tau \equiv \mathcal{M}$', color='tab:blue')
axs[1].loglog(np.arange(eq_coarse.n_m) + 1, np.mean(m_spectra_hdiff, axis=0), label=r'$\tau \equiv d(m)$', color='tab:orange')
axs[1].loglog(np.arange(eq_coarse.n_m) + 1, np.mean(m_spectra_0,     axis=0), label=r'$\tau \equiv 0$', color='tab:green')
axs[1].set_xlabel(r'$m + 1$', fontsize=15)
axs[1].set_ylabel(r'$E_T(m)$', fontsize=15)
axs[1].set_xlim((1, eq.n_m))
axs[1].legend(fontsize=13, frameon=False)
axs[1].tick_params(reset=True, axis='both', which='both', direction='in')

fig.tight_layout()
plt.show()

### Zonal strutures - radial profiles of the azimuthal velocity $u_{\varphi}(s)$ and potential vorticity $q(s)$:

In [ ]:
# Loading configuration
cfg_name = 'i'
cfg_data = os.path.join(save_path, cfg_name)
cfg_path = os.path.join('../data', cfg_name)
eq, *_ = QgAnnulus.load(os.path.join(cfg_path, 'snapshot.h5'))

# Dataset (and associated evaluation) name
data_name = 'continuous-turnover'
with h5py.File(os.path.join(cfg_path, data_name + '_dataset.h5'), 'r') as f:
    coarse_factor = f.attrs['coarse_factor']

eq_coarse = QgAnnulus(
    E=eq.E,
    cte_beta=eq.cte_beta,
    radius_ratio=eq.s_i / eq.s_o,
    n_m=int((eq.n_m - 1) / coarse_factor) + 1,
    n_s=int((eq.n_s - 1) / coarse_factor) + 1
)

def zonal_structures(
    name: str,
    file_path: str,
    eq: QgAnnulus
):
    comp_path = file_path + '.zonal_structures.npz'
    if os.path.isfile(comp_path):
        comp_data = np.load(comp_path)
        return (
            comp_data['mean_up_r'],
            comp_data['std_up_r'],
            comp_data['mean_q_r']
        )
    else:
        up_r = []
        q_r = []
        
        with h5py.File(file_path, 'r') as f:
            time = np.array(f['time'])
            samples = len(time)
            samples_digits = len(str(samples))
            print('Computing radial profiles of the azimuthal velocity and potential vorticity for {} ({} samples)'.format(name, str(samples)))
            pbar = tqdm.tqdm(range(samples), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
            for i in pbar:
                i_str = str(i).zfill(samples_digits)
                up_m = np.array(f['up_m_' + i_str])
                om_m = np.array(f['om_m_' + i_str])
                
                up_r.append(
                    np.mean(from_m(up_m, eq.n_phi), axis=0)
                )
                q_r.append(
                    (np.mean(from_m(om_m, eq.n_phi), axis=0) + 2 / eq.E) / eq.height
                )
                
        mean_up_r = np.mean(up_r, axis=0)
        std_up_r = np.std(up_r, axis=0)
        mean_q_r = np.mean(q_r, axis=0)
        np.savez(
            comp_path, 
            mean_up_r=mean_up_r, 
            std_up_r=std_up_r, 
            mean_q_r=mean_q_r
        )
        return (
            mean_up_r,
            std_up_r,
            mean_q_r,
        )

# Reference
path_dns = os.path.join(cfg_data, data_name + '_eval_dns.h5')
mean_up_r_dns, std_up_r_dns, mean_q_r_dns = zonal_structures(
    name='DNS', 
    file_path=path_dns, 
    eq=eq
)

# Models
path_learn = os.path.join(cfg_data, data_name + '_eval_learn.h5')
mean_up_r_learn, std_up_r_learn, mean_q_r_learn = zonal_structures(
    name='`Learned` model', 
    file_path=path_learn, 
    eq=eq_coarse
)

path_hdiff = os.path.join(cfg_data, data_name + '_eval_hdiff.h5')
mean_up_r_hdiff, std_up_r_hdiff, mean_q_r_hdiff = zonal_structures(
    name='`Hyperdiffusivity` model', 
    file_path=path_hdiff, 
    eq=eq_coarse
)

path_0 = os.path.join(cfg_data, data_name + '_eval_0.h5')
mean_up_r_0, std_up_r_0, mean_q_r_0 = zonal_structures(
    name='`Under-resolved` model',
    file_path=path_0,     
    eq=eq_coarse
)

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(7.5, 3.75), dpi=120)

axs[0].plot(eq.s_grid,        mean_up_r_dns,   label=r'$\text{DNS}$', color='k')
axs[0].plot(eq_coarse.s_grid, mean_up_r_learn, label=r'$\tau \equiv \mathcal{M}$', color='tab:blue')
axs[0].plot(eq_coarse.s_grid, mean_up_r_hdiff, label=r'$\tau \equiv d(m)$', color='tab:orange')
axs[0].plot(eq_coarse.s_grid, mean_up_r_0,     label=r'$\tau \equiv 0$', color='tab:green')

axs[0].fill_between(eq.s_grid,        mean_up_r_dns   - std_up_r_dns,   mean_up_r_dns   + std_up_r_dns,   alpha=0.2, color='k')
axs[0].fill_between(eq_coarse.s_grid, mean_up_r_learn - std_up_r_learn, mean_up_r_learn + std_up_r_learn, alpha=0.2, color='tab:blue')
axs[0].fill_between(eq_coarse.s_grid, mean_up_r_hdiff - std_up_r_hdiff, mean_up_r_hdiff + std_up_r_hdiff, alpha=0.2, color='tab:orange')
axs[0].fill_between(eq_coarse.s_grid, mean_up_r_0     - std_up_r_0,     mean_up_r_0     + std_up_r_0,     alpha=0.2, color='tab:green')

axs[0].ticklabel_format(style='scientific', scilimits=(0,0), useMathText=True)
axs[0].set_xlabel(r'$s$', fontsize=15)
axs[0].set_ylabel(r'$\langle u_\varphi \rangle_\varphi(s)$', fontsize=15)
axs[0].set_xlim((eq.s_i, eq.s_o))
axs[0].tick_params(reset=True, axis='both', which='both', direction='in')

axs[1].plot(eq.s_grid,        mean_q_r_dns,   label=r'$\text{DNS}$', color='k')
axs[1].plot(eq_coarse.s_grid, mean_q_r_learn, label=r'$\tau \equiv \mathcal{M}$', color='tab:blue')
axs[1].plot(eq_coarse.s_grid, mean_q_r_hdiff, label=r'$\tau \equiv d(m)$', color='tab:orange')
axs[1].plot(eq_coarse.s_grid, mean_q_r_0,     label=r'$\tau \equiv 0$', color='tab:green')
axs[0].axhline(0, color='darkgrey', linestyle='--')

axs[1].ticklabel_format(style='scientific', scilimits=(0,0), useMathText=True)
axs[1].set_xlabel(r'$s$', fontsize=15)
axs[1].set_ylabel(r'$\langle q \rangle_\varphi(s)$', fontsize=15)
axs[1].set_xlim((eq.s_i, eq.s_o))
axs[1].legend(fontsize=13, frameon=False)
axs[1].tick_params(reset=True, axis='both', which='both', direction='in')

fig.tight_layout()
plt.show()

### Spectral analysis - zonal and residual Bessel-Fourier spectra:

In [ ]:
# Loading configuration
cfg_name = 'ii'
cfg_data = os.path.join(save_path, cfg_name)
cfg_path = os.path.join('../data', cfg_name)
eq, *_ = QgAnnulus.load(os.path.join(cfg_path, 'snapshot.h5'))

# Dataset (and associated evaluation) name
data_name = 'continuous-turnover'
with h5py.File(os.path.join(cfg_path, data_name + '_dataset.h5'), 'r') as f:
    coarse_factor = f.attrs['coarse_factor']

eq_coarse = QgAnnulus(
    E=eq.E,
    cte_beta=eq.cte_beta,
    radius_ratio=eq.s_i / eq.s_o,
    n_m=int((eq.n_m - 1) / coarse_factor) + 1,
    n_s=int((eq.n_s - 1) / coarse_factor) + 1
)

print('Computing high-resolution grid Hankel roots and kernels...')
m_roots, kernels = hankel_kernels(eq)
k_weber_orr = m_roots[0, :]
dk = np.diff(k_weber_orr)
bins = np.zeros(len(k_weber_orr) + 1, np.float64)
bins[ 0] = k_weber_orr[ 0] - dk[ 0]/2
bins[-1] = k_weber_orr[-1] + dk[-1]/2
bins[1:-1] = k_weber_orr[:-1] + dk/2
dk = np.diff(bins)

print('Computing coarse-resolution grid Hankel roots and kernels...')
m_roots_coarse, kernels_coarse = hankel_kernels(eq_coarse)
k_weber_orr_coarse = m_roots_coarse[0, :]
dk_coarse = np.diff(k_weber_orr_coarse)
bins_coarse = np.zeros(len(k_weber_orr_coarse) + 1, np.float64)
bins_coarse[ 0] = k_weber_orr_coarse[ 0] - dk_coarse[ 0]/2
bins_coarse[-1] = k_weber_orr_coarse[-1] + dk_coarse[-1]/2
bins_coarse[1:-1] = k_weber_orr_coarse[:-1] + dk_coarse/2
dk_coarse = np.diff(bins_coarse)

def spectral_analysis(
    name: str,
    file_path: str,
    eq: QgAnnulus,
    m_roots: jnp.ndarray,
    kernels: jnp.ndarray,
    pad: bool
):
    comp_path = file_path + '.spectral_analysis.npz'
    if os.path.isfile(comp_path):
        comp_data = np.load(comp_path)
        return (
            comp_data['er_k'],
            comp_data['ez_k'],
        )
    else:
        er_k = []
        ez_k = []
        
        with h5py.File(file_path, 'r') as f:
            time = np.array(f['time'])
            samples = len(time)
            samples_digits = len(str(samples))
            print('Computing zonal and residual Bessel-Fourier spectra for {} ({} samples)'.format(name, str(samples)))
            pbar = tqdm.tqdm(range(samples), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
            for i in pbar:
                i_str = str(i).zfill(samples_digits)
                us_m = np.array(f['us_m_' + i_str])
                up_m = np.array(f['up_m_' + i_str])

                if pad:
                    us_m = np.pad(us_m, ((0, eq.n_m - us_m.shape[0]), (0, 0)))
                    us_r = jax.vmap(coef_r)(us_m)
                    us_r = 1 / np.sqrt(1 / coarse_factor) * np.pad(us_r, ((0, 0), (0, eq.n_s - us_r.shape[1])))
                    us_m = jax.vmap(coef_r)(us_r)
                    up_m = np.pad(up_m, ((0, eq.n_m - up_m.shape[0]), (0, 0)))
                    up_r = jax.vmap(coef_r)(up_m)
                    up_r = 1 / np.sqrt(1 / coarse_factor) * np.pad(up_r, ((0, 0), (0, eq.n_s - up_r.shape[1])))
                    up_m = jax.vmap(coef_r)(up_r)
    
                if i % (samples // 5) == 0:
                    us_k = hankel_spectrum(eq, m_roots, kernels, us_m)
                    up_k = hankel_spectrum(eq, m_roots, kernels, up_m)
                    if pad:
                        us_k = us_k[:eq_coarse.n_m, :eq_coarse.n_s]
                        up_k = up_k[:eq_coarse.n_m, :eq_coarse.n_s]
                    er_k.append(
                        (us_k + up_k)[1:] / eq.surf
                    )
                us_0 = hankel_spectrum(eq, m_roots, kernels, us_m, n_max_m=1)[0]
                up_0 = hankel_spectrum(eq, m_roots, kernels, up_m, n_max_m=1)[0]
                if pad:
                    us_0 = us_0[:eq_coarse.n_s]
                    up_0 = up_0[:eq_coarse.n_s]
                ez_k.append(
                    (us_0 + up_0) / eq.surf
                )
        np.savez(
            comp_path, 
            er_k=er_k, 
            ez_k=ez_k, 
        )
        return (
            np.array(er_k),
            np.array(ez_k)
        )

def residual_binning(
    er_k: np.ndarray,
    m_roots: np.ndarray,
    bins: np.ndarray,
    dk: np.ndarray
) -> np.ndarray:
    return [
        scipy.stats.binned_statistic(m_roots[1:].flatten(), er_k[i].flatten(), statistic='sum', bins=bins)[0] / dk 
        for i in range(len(er_k))
    ]

# Reference
path_dns = os.path.join(cfg_data, data_name + '_eval_dns.h5')
er_k_dns, ez_k_dns = spectral_analysis(
    name='DNS', 
    file_path=path_dns, 
    eq=eq,
    m_roots=m_roots,
    kernels=kernels,
    pad=False
)
er_k_dns = residual_binning(
    er_k_dns, m_roots, bins, dk
)

# Models
path_learn = os.path.join(cfg_data, data_name + '_eval_learn.h5')
er_k_learn, ez_k_learn = spectral_analysis(
    name='`Learned` model', 
    file_path=path_learn, 
    eq=eq,
    m_roots=m_roots,
    kernels=kernels,
    pad=True
)
er_k_learn = residual_binning(
    er_k_learn, m_roots_coarse, bins_coarse, dk_coarse
)

path_hdiff = os.path.join(cfg_data, data_name + '_eval_hdiff.h5')
er_k_hdiff, ez_k_hdiff = spectral_analysis(
    name='`Hyperdiffusivity` model', 
    file_path=path_hdiff, 
    eq=eq,
    m_roots=m_roots,
    kernels=kernels,
    pad=True
)
er_k_hdiff = residual_binning(
    er_k_hdiff, m_roots_coarse, bins_coarse, dk_coarse
)

path_0 = os.path.join(cfg_data, data_name + '_eval_0.h5')
er_k_0, ez_k_0 = spectral_analysis(
    name='`Under-resolved` model',
    file_path=path_0,
    eq=eq,
    m_roots=m_roots,
    kernels=kernels,
    pad=True
)
er_k_0 = residual_binning(
    er_k_0, m_roots_coarse, bins_coarse, dk_coarse
)

fig, axs = plt.subplots(ncols=1, nrows=1, figsize=(3.75, 3.75), dpi=120)

axs.loglog(bins[1:] - dk/2,               np.mean(er_k_dns,   axis=0), label=r'$\text{DNS}$', color='k')
axs.loglog(bins_coarse[1:] - dk_coarse/2, np.mean(er_k_learn, axis=0), label=r'$\tau \equiv \mathcal{M}$', color='tab:blue')
axs.loglog(bins_coarse[1:] - dk_coarse/2, np.mean(er_k_hdiff, axis=0), label=r'$\tau \equiv d(m)$', color='tab:orange')
axs.loglog(bins_coarse[1:] - dk_coarse/2, np.mean(er_k_0,     axis=0), label=r'$\tau \equiv 0$', color='tab:green')

axs.loglog(m_roots[0],        np.mean(ez_k_dns,   axis=0), linestyle='--', alpha=0.5, color='k', )
axs.loglog(m_roots_coarse[0], np.mean(ez_k_learn, axis=0), linestyle='--', alpha=0.5, color='tab:blue')
axs.loglog(m_roots_coarse[0], np.mean(ez_k_hdiff, axis=0), linestyle='--', alpha=0.5, color='tab:orange')
axs.loglog(m_roots_coarse[0], np.mean(ez_k_0,     axis=0), linestyle='--', alpha=0.5, color='tab:green')

axs.set_xlabel(r'$k$', fontsize=15)
axs.set_ylabel(r'$\text{Kinetic \,\, energy \,\, spectra}$', fontsize=15)
axs.set_xlim((m_roots[0, 0], m_roots[0, -1]))
axs.legend(fontsize=13, frameon=False, handlelength=1)
axs.tick_params(reset=True, axis='both', which='both', direction='in')

fig.tight_layout()
plt.show()

### Temporal patterns - Hovmöller maps of the azimuthal and radial velocities:

In [ ]:
# Loading configuration
cfg_name = 'ii'
cfg_data = os.path.join(save_path, cfg_name)
cfg_path = os.path.join('../data', cfg_name)
eq, *_ = QgAnnulus.load(os.path.join(cfg_path, 'snapshot.h5'))

# Dataset (and associated evaluation) name
data_name = 'continuous-turnover'
with h5py.File(os.path.join(cfg_path, data_name + '_dataset.h5'), 'r') as f:
    coarse_factor = f.attrs['coarse_factor']

eq_coarse = QgAnnulus(
    E=eq.E,
    cte_beta=eq.cte_beta,
    radius_ratio=eq.s_i / eq.s_o,
    n_m=int((eq.n_m - 1) / coarse_factor) + 1,
    n_s=int((eq.n_s - 1) / coarse_factor) + 1
)

def temporal_patterns(
    name: str,
    file_path: str,
    eq: QgAnnulus,
    radial_idx: int
):
    comp_path = file_path + '.temporal_patterns.npz'
    if os.path.isfile(comp_path):
        comp_data = np.load(comp_path)
        return (
            comp_data['time'],
            comp_data['up_ravg'],
            comp_data['us_pavg']
        )
    else:
        up_ravg = []
        us_pavg = []
        
        with h5py.File(file_path, 'r') as f:
            time = np.array(f['time'])
            samples = len(time)
            samples_digits = len(str(samples))
            print('Computing radial/azimuthal profiles of the azimuthal/radial velocities for {} ({} samples)'.format(name, str(samples)))
            pbar = tqdm.tqdm(range(samples), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
            for i in pbar:
                i_str = str(i).zfill(samples_digits)
                us_m = np.array(f['us_m_' + i_str])
                up_m = np.array(f['up_m_' + i_str])
    
                up_ravg.append(
                    np.mean(from_m(up_m, eq.n_phi), axis=0)
                )
                us_pavg.append(
                    from_m(us_m, eq.n_phi)[:, radial_idx]
                )
        np.savez(
            comp_path, 
            time=time, 
            up_ravg=up_ravg, 
            us_pavg=us_pavg
        )
        return (
            time,
            np.array(up_ravg),
            np.array(us_pavg)
        )

# Reference
path_dns = os.path.join(cfg_data, data_name + '_eval_dns.h5')
time_dns, up_ravg_dns, us_pavg_dns = temporal_patterns(
    name='DNS', 
    file_path=path_dns, 
    eq=eq,
    radial_idx=100
)

# Model
path_learn = os.path.join(cfg_data, data_name + '_eval_learn.h5')
time_learn, up_ravg_learn, us_pavg_learn = temporal_patterns(
    name='`Learned` model', 
    file_path=path_learn, 
    eq=eq_coarse,
    radial_idx=20
)

fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(7.5, 5.0), constrained_layout=True)

up_r_mag = 0.4*np.max(np.abs(up_ravg_dns))

cbar = axs[0].contourf(
    time_dns, 
    eq.s_grid, 
    up_ravg_dns.T, 
    levels=50, vmin=-up_r_mag, vmax=up_r_mag, cmap='RdBu_r', extend='both'
)
axs[1].contourf(
    time_learn, 
    eq_coarse.s_grid, 
    up_ravg_learn.T, 
    levels=50, vmin=-up_r_mag, vmax=up_r_mag, cmap='RdBu_r', extend='both'
)

axs[0].set_title(r'$(a) \,\, \text{DNS}$', fontsize=12, loc='left')
axs[0].set_ylabel(r'$s$', fontsize=15)
axs[1].set_title(r'$(b) \,\, \tau \equiv \mathcal{M}$', fontsize=12, loc='left')
axs[1].set_ylabel(r'$s$', fontsize=15)
axs[1].set_xlabel(r'$t$', fontsize=15)

x, y = 0.16, -0.03
width = 0.75
height = 0.02
cbar_axs = fig.add_axes([x, y, width, height])
cbar_axs.set_in_layout(False)
fig.colorbar(cbar, cax=cbar_axs, orientation='horizontal', aspect=5, shrink=0.2)

fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(7.5, 5.0), constrained_layout=True)

us_p_mag = 0.35*np.max(np.abs(us_pavg_dns))

# Only show 1 turnover time
turnover_time = 1.7e-4

start_idx_dns = np.argmin(np.abs(time_dns - time_learn[0]))
end_idx_dns = np.argmin(np.abs(time_dns - (time_learn[0] + turnover_time)))
cbar = axs[0].contourf(
    np.linspace(0, 2 * np.pi, eq.n_phi, endpoint=False), 
    time_dns[start_idx_dns:end_idx_dns], 
    us_pavg_dns[start_idx_dns:end_idx_dns], 
    levels=50, vmin=-us_p_mag, vmax=us_p_mag, cmap='PuOr', extend='both'
)

end_idx_learn = np.argmin(np.abs(time_learn - (time_learn[0] + turnover_time)))
axs[1].contourf(
    np.linspace(0, 2 * np.pi, eq_coarse.n_phi, endpoint=False), 
    time_learn[:end_idx_learn], 
    us_pavg_learn[:end_idx_learn], 
    levels=50, vmin=-us_p_mag, vmax=us_p_mag, cmap='PuOr', extend='both'
)

axs[0].set_title(r'$(a) \,\, \text{DNS}$', fontsize=12, loc='left')
axs[0].set_ylabel(r'$t$', fontsize=15)
axs[0].set_yticks(np.linspace(0.010175, 0.010300, 6))
axs[1].set_title(r'$(b) \,\, \tau \equiv \mathcal{M}$', fontsize=12, loc='left')
axs[1].set_ylabel(r'$t$', fontsize=15)
axs[1].set_xlabel(r'$\varphi$', fontsize=15)
axs[1].set_yticks(np.linspace(0.010175, 0.010300, 6))

x, y = 0.18, -0.03
width = 0.75
height = 0.02
cbar_axs = fig.add_axes([x, y, width, height])
fig.colorbar(cbar, cax=cbar_axs, orientation='horizontal', aspect=5, shrink=0.2)
plt.show()

### Integrated quantities:

In [ ]:
def forcing_prep(
    eq: QgAnnulus,
    dx_f: float,
) -> np.ndarray:
    """
    Cartesian forcing described in
    
    Zonal jets experiments in the gas giants’ zonostrophic regime.
    D. Lemasquerier, B. Favier and M. Le Bars.
    Icarus 390 (2023).
    """
    nx = int(2 * eq.s_o / dx_f + 1)
    ny = nx
    dx = 2 * eq.s_o / (nx - 1)
    dy = dx
    
    x_lins, y_lins = np.meshgrid(np.arange(nx), np.arange(ny), indexing='ij')
    amp_grid = (-1)**(x_lins + 1) * (-1)**(y_lins + 1)
    x_grid, y_grid = np.meshgrid(-eq.s_o + dx * np.arange(nx), -eq.s_o + dy * np.arange(ny), indexing='ij')
    iso_grid = np.sqrt(x_grid*x_grid + y_grid*y_grid)
    
    pump_position = (iso_grid >= eq.s_i + 0.5 * dx) & \
                    (iso_grid <= eq.s_o - 0.5 * dx)
    
    amp = amp_grid[pump_position]
    x = x_grid[pump_position]
    y = y_grid[pump_position]
    return x, y, amp

dx_f = 0.08
radius_f = 0.04
a_f = 2e10
xpump, ypump, amppump = forcing_prep(eq, dx_f)

def integrated_quantities(
    name: str,
    file_path: str,
    eq: QgAnnulus
):
    comp_path = file_path + '.integrated_quantities.npz'
    if os.path.isfile(comp_path):
        comp_data = np.load(comp_path)
        return (
            comp_data['re'],
            comp_data['ez_ratio'],
            comp_data['ens'],
            comp_data['diss_l'],
            comp_data['power'],
            comp_data['v_diss'],
            comp_data['f_diss']
        )
    else:
        re = []
        ez_ratio = []
        ens = []
        diss_l = []
        power = []
        v_diss = []
        f_diss = []

        rr, pp = np.meshgrid(eq.s_grid, 2 * np.pi * np.arange(eq.n_phi) / eq.n_phi)
        n_pumps = len(xpump)
        
        with h5py.File(file_path, 'r') as f:
            time = np.array(f['time'])
            samples = len(time)
            samples_digits = len(str(samples))
            print('Computing integrated quantities for {} ({} samples)'.format(name, str(samples)))
            pbar = tqdm.tqdm(range(samples), bar_format='{l_bar}{bar:10}{r_bar}{bar:-10b}')
            for i in pbar:
                i_str = str(i).zfill(samples_digits)
                us_m = np.array(f['us_m_' + i_str])
                up_m = np.array(f['up_m_' + i_str])
                om_m = np.array(f['om_m_' + i_str])

                et = 0.5 * (average(eq, us_m) + average(eq, up_m))
                ez = 0.5 * average(eq, up_m[[0]])

                up = from_m(up_m, eq.n_phi)
                us = from_m(us_m, eq.n_phi)

                xx = rr * np.cos(pp)
                yy = rr * np.sin(pp)

                ux = us * np.cos(pp) - up * np.sin(pp)
                uy = us * np.sin(pp) + up * np.cos(pp)

                def __pump_power__(xpump, ypump, amppump):
                    si = jnp.sqrt((xx - xpump)**2 + (yy - ypump)**2)
                    return amppump / si**2 * (1 - jnp.exp(-si**2 / radius_f**2)) * (-(yy - ypump)*ux + (xx - xpump)*uy)

                fdotu = 0.5 * a_f * radius_f**2 * np.sum(jax.vmap(__pump_power__)(xpump, ypump, amppump), axis=0)
                fdotu_tot = 2 * np.pi * np.mean(fdotu, axis=0)
                fpower = np.real(quad_r(fdotu_tot * eq.s_grid)) / eq.surf
                
                re.append(
                    reynolds(eq, us_m, up_m)
                )
                ez_ratio.append(
                    ez / et
                )
                ens.append(
                    0.5 * average(eq, om_m)
                )
                diss_l.append(
                    np.sqrt(et / ens[-1])
                )
                
                power.append(
                    fpower
                )
                v_diss.append(
                    2 * ens[-1]
                )
                f_diss.append(
                    fpower - v_diss[-1]
                )
        np.savez(
            comp_path, 
            re=re, 
            ez_ratio=ez_ratio, 
            ens=ens,
            diss_l=diss_l,
            power=power,
            v_diss=v_diss,
            f_diss=f_diss
        )
        return (
            np.array(re),
            np.array(ez_ratio),
            np.array(ens),
            np.array(diss_l),
            np.array(power),
            np.array(v_diss),
            np.array(f_diss)
        )

configs = [
    ('i', 'continuous-turnover'), 
    ('ii', 'continuous-turnover'), 
    ('iii', 'continuous-turnover')
]

fig, axs = plt.subplots(ncols=1, nrows=len(configs), dpi=120)

row_labels = ['DNS', '`Learned` model', '`Hyperdiffusivity` model', '`Under-resolved` model']
col_labels = [r'$Re$', r'$E_Z / E_T$', r'$\mathcal{Z}$', r'$\ell_\nu$', r'$\mathcal{P}_\mathcal{F}$', r'$\alpha_\nu^\Upsilon$']

for i, (cfg_name, data_name) in enumerate(configs):
    cfg_data = os.path.join(save_path, cfg_name)
    cfg_path = os.path.join('../data', cfg_name)
    eq, *_ = QgAnnulus.load(os.path.join(cfg_path, 'snapshot.h5'))
    
    # Dataset (and associated evaluation) name
    with h5py.File(os.path.join(cfg_path, data_name + '_dataset.h5'), 'r') as f:
        coarse_factor = f.attrs['coarse_factor']
    print('Computing integrated quantities for config ({})'.format(cfg_name))

    eq_coarse = QgAnnulus(
        E=eq.E,
        cte_beta=eq.cte_beta,
        radius_ratio=eq.s_i / eq.s_o,
        n_m=int((eq.n_m - 1) / coarse_factor) + 1,
        n_s=int((eq.n_s - 1) / coarse_factor) + 1
    )

    # Reference
    path_dns = os.path.join(cfg_data, data_name + '_eval_dns.h5')
    re_dns, ez_ratio_dns, ens_dns, diss_l_dns, power_dns, v_diss_dns, f_diss_dns = integrated_quantities(
        name='DNS', 
        file_path=path_dns, 
        eq=eq
    )

    # Models
    path_learn = os.path.join(cfg_data, data_name + '_eval_learn.h5')
    re_learn, ez_ratio_learn, ens_learn, diss_l_learn, power_learn, v_diss_learn, f_diss_learn = integrated_quantities(
        name='`Learned` model', 
        file_path=path_learn, 
        eq=eq_coarse
    )

    path_hdiff = os.path.join(cfg_data, data_name + '_eval_hdiff.h5')
    re_hdiff, ez_ratio_hdiff, ens_hdiff, diss_l_hdiff, power_hdiff, v_diss_hdiff, f_diss_hdiff = integrated_quantities(
        name='`Hyperdiffusivity` model', 
        file_path=path_hdiff, 
        eq=eq_coarse
    )
    
    path_0 = os.path.join(cfg_data, data_name + '_eval_0.h5')
    re_0, ez_ratio_0, ens_0, diss_l_0, power_0, v_diss_0, f_diss_0 = integrated_quantities(
        name='`Under-resolved` model',
        file_path=path_0,
        eq=eq_coarse
    )

    f_ratio_dns = np.mean(f_diss_dns) / (np.mean(f_diss_dns) + np.mean(v_diss_dns))
    f_ratio_learn = np.mean(f_diss_learn) / (np.mean(f_diss_learn) + np.mean(v_diss_learn))
    f_ratio_hdiff = np.mean(f_diss_hdiff) / (np.mean(f_diss_hdiff) + np.mean(v_diss_hdiff))
    f_ratio_0   = np.mean(f_diss_0) / (np.mean(f_diss_0) + np.mean(v_diss_0))
    cell_values = np.stack((
        ['%.0f' % np.mean(re_dns)  , '%.3f' % np.mean(ez_ratio_dns),   '%.2e' % np.mean(ens_dns),   
         '%.2e' % np.mean(diss_l_dns),   '%.2e' % np.mean(power_dns),   '%.3f' % np.mean(f_ratio_dns)], 
        ['%.0f' % np.mean(re_learn), '%.3f' % np.mean(ez_ratio_learn), '%.2e' % np.mean(ens_learn), 
         '%.2e' % np.mean(diss_l_learn), '%.2e' % np.mean(power_learn), '%.3f' % np.mean(f_ratio_learn)],
        ['%.0f' % np.mean(re_hdiff), '%.3f' % np.mean(ez_ratio_hdiff), '%.2e' % np.mean(ens_hdiff), 
         '%.2e' % np.mean(diss_l_hdiff), '%.2e' % np.mean(power_hdiff), '%.3f' % np.mean(f_ratio_hdiff)],
        ['%.0f' % np.mean(re_0)    , '%.3f' % np.mean(ez_ratio_0),     '%.2e' % np.mean(ens_0),     
         '%.2e' % np.mean(diss_l_0),     '%.2e' % np.mean(power_0),     '%.3f' % np.mean(f_ratio_0)]
    ))

    cfg_ax = axs[i] if len(configs) > 1 else axs
    cfg_ax.axis('off')
    cfg_ax.table(cellText=cell_values, rowLabels=row_labels, colLabels=col_labels, loc='top', cellLoc='left', fontsize=50)
    cfg_ax.text(-0.4, 1.7, r'$\text{Config \,\, ' + '(' + cfg_name + ')}$')
plt.show()